In [9]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras.backend as K
import keras
from keras import optimizers
# from keras.engine.topology import Layer
from keras.layers import Activation, Lambda, Conv1D, SpatialDropout1D, Dense, BatchNormalization
from keras.models import Input, Model
# import horovod.tensorflow.keras as hvd

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [18]:
def gen_seq(data, feat_len, pre_len):
    x, y = list(), list()
    for i in range(data.shape[0]-(feat_len+pre_len)+1):
        for j in range(data.shape[1]):
            x.append(data.iloc[i:i+feat_len,j])
            y.append(data.iloc[i+feat_len,j])
    return np.asarray(x), np.asarray(y)

In [20]:
# print(x.shape, y.shape)
# %timeit x, y = gen_seq(df2, 4, 1)
# X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, 
#                                                     random_state=42)

In [12]:
# df2 = pd.read_csv('./PeMSD7_V_228.csv', header=None)
df2.shape

(12672, 228)

In [ ]:
class TCN:
    # Compile model
    def build(self):
        batch_size, timesteps, input_dim = None, 4, 1

        i = Input(batch_shape=(batch_size, timesteps, input_dim))
        o = tcn_layer(nb_filters=128,
                      kernel_size=4,
                      dilations=(1,2,4,8,16,32,64),
                      padding='causal',
                      dropout_rate=0.01,
                      return_sequences=False,
                      kernel_initializer='he_normal',
                      use_batch_norm=True)(i)
        o = Dense(1)(o)

        self.m = Model(inputs=[i], outputs=[o])
        return self.m
#         self.m.compile(optimizer='adam',loss='hinge', metrics=['accuracy',precision_threshold(0),recall_threshold(0)])


    # Helper function
    # The output should be in the form of 1 and -1 given the hinge loss function
    def reshape(self, y_train):
        for i in range(y_train.shape[0]):
            if y_train[i] == 0:
                y_train[i] = -1
        return y_train

    # Train model
    def train(self, X_train, y_train):
        self.num_epochs = 1 
        y_train = self.reshape(y_train)

        # Directory where the checkpoints will be saved
        checkpoint_dir = '/Users/zhiyun/Desktop/Fall19-20/TEMG4000/startup_prediction/src/model'
        checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_1")

        checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,
                                                               save_weights_only=True,
                                                               save_freq=1024)

        self.history = self.m.fit(X_train, y_train, epochs=self.num_epochs, validation_split=0.25, callbacks=[checkpoint_callback])

    # Predict test data
    def predict(self, X_test):
        return self.m.predict(X_test)
    
    # Evaluate model performance with metrics
    def evaluate(self, y_pred, y_test):
        y_pred_rounded = np.clip(y_pred, 0, 1)
        for i in range(y_pred_rounded.shape[0]):
            if y_pred_rounded[i] > 0:
                y_pred_rounded[i] = 1

        precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred_rounded)                                                                                                                                                                                                          
        print('Precision:', precision.mean())
        print('Recall:', recall.mean())
        print('f1-score:', f1.mean())
        
        print('Classification report:')
        print(classification_report(y_test, y_pred_rounded))

        print('Confusion matrix:')
        print(confusion_matrix(y_test, y_pred_rounded))

    # Save model
    def save_model(self):
        self.m.save('src/model/tcn.hdf5')